In [1]:
import pandas as pd

In [38]:
bea_gdp_df = pd.read_csv('./data/bea/GDP_2020-2009.csv',dtype={'GeoFips':'string', 'year':'Int64'})
bea_gdp_df.dropna(inplace=True)
county_GEOID_cross_df = pd.read_csv('./data/crosswalk_county_name_to_county_fips.csv',dtype={'GEOID':'string'})
county_GEOID_cross_df.shape

(3234, 5)

In [52]:
unique_bea_geofips = bea_gdp_df.groupby('GeoFips').sum().reset_index()
unique_county_GEOIDS = county_GEOID_cross_df.groupby('GEOID').sum().reset_index()

In [60]:
# How many missing values are there in the county_GEOID_cross_df?
# That is to say, how many rows in bea are missing rows in the county crosswalk?
test_merge = pd.merge(unique_bea_geofips, unique_county_GEOIDS,
                      left_on = ['GeoFips'],
                      right_on = ['GEOID'],
                      how = 'outer',
                      indicator = 'matched',
                      validate = 'one_to_one')

print(len(test_merge.query("matched == 'left_only'").GeoFips.unique()))
# The missing values
missing_values_bea = pd.DataFrame(test_merge.query("matched == 'left_only'").groupby(['GeoFips']).size()).reset_index()['GeoFips'].values
missing_values_crosswalk = pd.DataFrame(test_merge.query("matched == 'right_only'").groupby(['GEOID']).size()).reset_index()['GEOID'].values

28


In [62]:
print(missing_values_crosswalk)

<StringArray>
['15005', '15009', '51003', '51005', '51015', '51031', '51035', '51053',
 '51059', '51069',
 ...
 '72141', '72143', '72145', '72147', '72149', '72151', '72153', '78010',
 '78020', '78030']
Length: 144, dtype: string


Let's check to see how many of the missing values are in the patents data. Those are those that we need to worry about.

In [55]:
patents = pd.read_csv('./data/patentsview/all_patents_granted_and pregrant.csv',dtype={'GEOID':'string'}, low_memory=False)
unique_patent_GEOIDS = patents.groupby('GEOID').sum().reset_index()
unique_patent_GEOIDS_list = unique_patent_GEOIDS.GEOID.values.tolist()

In [66]:
patents.query("GEOID == '51790'")  


,patent_number,assignee,grant_year,application_year,application_number,GEOID,ipc_section,team_size,inventors,men_inventors,women_inventors,already_granted
1426376,7740267,"ARCHIMEDES INTELLECTUAL, LLC",2010.0,2008,12078682,51790,B F,1,1,1.0,0,1
1426377,9581257,CLEAN PLANET INC.,2017.0,2013,13901202,51790,F,1,1,1.0,0,1
1426378,10401117,"PARKER COMPOUND BOWS, INC.",2019.0,2018,16039548,51790,F,2,2,2.0,0,1
1578573,NaN,"Archimedes Intellectual, LLC",NaN,2009,12385188,51790,NaN,1,1,1.0,0,0
1960143,NaN,CLEAN PLANET Inc.,NaN,2013,13901202,51790,NaN,1,1,1.0,0,0


In [56]:
# See how many of the missing values are in the patents dataframe.
values_in_bea_not_in_patents = list(set(missing_values_bea) - set(unique_patent_GEOIDS_list))
print(len(values_in_bea_not_in_patents))

28


In [58]:
# Check against the original missing values list.
values_in_bea_we_need_to_fix = list(set(missing_values_bea) - set(values_in_bea_not_in_patents))
print(len(values_in_bea_we_need_to_fix))
values_in_bea_we_need_to_fix

0


[]

Check it the other way? How many GEOIDS in the patent data are missing from the bea data?

In [ ]:
# See how many of the missing values are in the patents dataframe.
values_in_bea_not_in_patents = list(set(missing_values) - set(unique_patent_GEOIDS_list))
print(len(values_in_bea_not_in_patents))